# Unifier Exogenous Feature Selection

Analyze all Unifier exogenous features (levels + MoM diffs + z-scores) using VIF to determine which transformations provide unique explanatory power and prune multicollinearity.

In [32]:
import pandas as pd
import numpy as np
from pathlib import Path
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)

## 1. Load Snapshot & Pivot to Wide Format

In [33]:
# Find the latest available snapshot
base_dir = Path('data/Exogenous_data/exogenous_unifier_data')
all_snaps = sorted(base_dir.glob('**/*.parquet'))
print(f'Total snapshots: {len(all_snaps)}')
print(f'Latest: {all_snaps[-1]}')

# Load a recent snapshot (prefer 2025-12, fallback to latest)
snap_path = base_dir / 'decades' / '2020s' / '2025' / '2025-12.parquet'
if not snap_path.exists():
    snap_path = all_snaps[-1]
    print(f'Using fallback: {snap_path}')

df = pd.read_parquet(snap_path)
print(f'\nRaw shape: {df.shape}')
print(f'Columns: {df.columns.tolist()}')
print(f'Unique series: {df["series_name"].nunique()}')
print(f'\nAll series names:')
for s in sorted(df['series_name'].unique()):
    print(f'  {s}')

Total snapshots: 433
Latest: data/Exogenous_data/exogenous_unifier_data/decades/2020s/2026/2026-02.parquet

Raw shape: (24546, 6)
Columns: ['date', 'release_date', 'series_code', 'snapshot_date', 'value', 'series_name']
Unique series: 41

All series names:
  AHE_Private
  AHE_Private_diff
  AHE_Private_diff_zscore_12m
  AHE_Private_diff_zscore_3m
  AWH_All_Private
  AWH_All_Private_diff
  AWH_All_Private_diff_zscore_3m
  AWH_Manufacturing
  AWH_Manufacturing_diff_zscore_12m
  AWH_Manufacturing_diff_zscore_3m
  CB_Consumer_Confidence_diff
  CB_Consumer_Confidence_diff_zscore_3m
  Challenger_Job_Cuts
  Challenger_Job_Cuts_diff
  Challenger_Job_Cuts_diff_zscore_12m
  Challenger_Job_Cuts_diff_zscore_3m
  Empire_State_Mfg
  Empire_State_Mfg_diff
  Empire_State_Mfg_diff_zscore_12m
  Empire_State_Mfg_diff_zscore_3m
  Housing_Starts
  Housing_Starts_diff
  Housing_Starts_diff_zscore_12m
  Housing_Starts_diff_zscore_3m
  ISM_Manufacturing_Index
  ISM_Manufacturing_Index_diff
  ISM_Manufacturing

In [34]:
# Pivot to wide format
wide = df.pivot_table(index='date', columns='series_name', values='value', aggfunc='first')
wide = wide.sort_index()
print(f'Wide shape: {wide.shape}')
print(f'Date range: {wide.index.min()} to {wide.index.max()}')
print(f'\nNaN counts per feature:')
print(wide.isna().sum().sort_values(ascending=False))

Wide shape: (1284, 41)
Date range: 1919-01-01 00:00:00 to 2025-12-01 00:00:00

NaN counts per feature:
series_name
AWH_All_Private_diff_zscore_3m                1076
AHE_Private_diff_zscore_12m                   1054
AHE_Private_diff_zscore_3m                    1050
AHE_Private_diff                              1049
AHE_Private                                   1048
AWH_All_Private_diff                          1048
AWH_All_Private                               1047
Empire_State_Mfg_diff_zscore_12m               996
Empire_State_Mfg_diff_zscore_3m                992
Empire_State_Mfg_diff                          991
Empire_State_Mfg                               990
ISM_NonManufacturing_Index_diff_zscore_12m     948
ISM_NonManufacturing_Index_diff_zscore_3m      944
ISM_NonManufacturing_Index                     942
Challenger_Job_Cuts_diff_zscore_12m            906
Challenger_Job_Cuts_diff_zscore_3m             902
Challenger_Job_Cuts_diff                       901
Challenger_Job_Cut

## 2. Define Feature Groups

In [35]:
# Base series names (12 series)
BASE_SERIES = [
    'AHE_Private', 'AWH_All_Private', 'AWH_Manufacturing',
    'CB_Consumer_Confidence', 'Challenger_Job_Cuts',
    'Empire_State_Mfg', 'Housing_Starts',
    'ISM_Manufacturing_Index', 'ISM_NonManufacturing_Index',
    'Industrial_Production', 'Retail_Sales', 'UMich_Expectations',
]

# Group by series
SERIES_GROUPS = {}
for base in BASE_SERIES:
    SERIES_GROUPS[base] = [c for c in wide.columns if c == base or c.startswith(f'{base}_diff')]

# Group by transformation type
TRANSFORM_GROUPS = {
    'level': [c for c in wide.columns if not c.endswith('_diff') and 
              not c.endswith('_diff_zscore_12m') and not c.endswith('_diff_zscore_3m')],
    'diff': [c for c in wide.columns if c.endswith('_diff')],
    'diff_zscore_12m': [c for c in wide.columns if c.endswith('_diff_zscore_12m')],
    'diff_zscore_3m': [c for c in wide.columns if c.endswith('_diff_zscore_3m')],
}

print('Feature counts by series group:')
for group, feats in SERIES_GROUPS.items():
    print(f'  {group}: {len(feats)} features')

print(f'\nFeature counts by transformation:')
for ttype, feats in TRANSFORM_GROUPS.items():
    print(f'  {ttype}: {len(feats)} features')

print(f'\nTotal features: {len(wide.columns)}')

Feature counts by series group:
  AHE_Private: 4 features
  AWH_All_Private: 3 features
  AWH_Manufacturing: 3 features
  CB_Consumer_Confidence: 2 features
  Challenger_Job_Cuts: 4 features
  Empire_State_Mfg: 4 features
  Housing_Starts: 4 features
  ISM_Manufacturing_Index: 3 features
  ISM_NonManufacturing_Index: 3 features
  Industrial_Production: 4 features
  Retail_Sales: 4 features
  UMich_Expectations: 3 features

Feature counts by transformation:
  level: 11 features
  diff: 9 features
  diff_zscore_12m: 9 features
  diff_zscore_3m: 12 features

Total features: 41


## 3. VIF Helper Functions

In [36]:
def compute_group_vif(df_wide, feature_list, group_name, min_obs=50):
    """Compute VIF for a group of features."""
    cols = [c for c in feature_list if c in df_wide.columns]
    if len(cols) < 2:
        return pd.DataFrame()
    
    X = df_wide[cols].replace([np.inf, -np.inf], np.nan).dropna()
    
    # Drop zero-variance columns
    zero_var = X.columns[X.var() == 0]
    if len(zero_var) > 0:
        print(f'  Dropping {len(zero_var)} zero-variance cols: {list(zero_var)}')
        X = X.loc[:, X.var() > 0]
    
    if X.shape[1] < 2:
        return pd.DataFrame()
    
    X_const = X.copy()
    X_const['_const'] = 1.0
    
    vif_data = []
    for col in [c for c in X_const.columns if c != '_const']:
        try:
            vif = variance_inflation_factor(X_const.values, X_const.columns.get_loc(col))
            vif_data.append({'group': group_name, 'feature': col, 'VIF': round(vif, 2)})
        except Exception:
            vif_data.append({'group': group_name, 'feature': col, 'VIF': np.nan})
    
    return pd.DataFrame(vif_data).sort_values('VIF', ascending=False)


def iterative_vif_pruning(df_wide, features, threshold=5.0, protected=None, verbose=True):
    """Iteratively remove highest-VIF feature until all below threshold."""
    protected = protected or set()
    cols = [c for c in features if c in df_wide.columns]
    X = df_wide[cols].replace([np.inf, -np.inf], np.nan).dropna()
    X = X.loc[:, X.var() > 0]
    removed = []
    
    iteration = 0
    while len(X.columns) > 2:
        iteration += 1
        X_const = X.copy()
        X_const['_const'] = 1.0
        
        vifs = {}
        for col in [c for c in X_const.columns if c != '_const']:
            try:
                vifs[col] = variance_inflation_factor(X_const.values, X_const.columns.get_loc(col))
            except:
                vifs[col] = np.inf
        
        non_protected_vifs = {k: v for k, v in vifs.items() if k not in protected}
        if not non_protected_vifs:
            break
        
        max_col = max(non_protected_vifs, key=non_protected_vifs.get)
        max_vif = non_protected_vifs[max_col]
        
        if max_vif <= threshold:
            break
        
        X = X.drop(columns=[max_col])
        removed.append((max_col, round(max_vif, 2)))
        if verbose:
            print(f'  Iter {iteration}: Removed {max_col} (VIF={max_vif:.1f})')
    
    return X.columns.tolist(), removed

## 4. Transformation Correlation Analysis

For each base series, how correlated are the different transformations?

In [37]:
# Level vs Diff correlation for each base series
correlation_analysis = []

for base in BASE_SERIES:
    diff_feat = f'{base}_diff'
    z12_feat = f'{base}_diff_zscore_12m'
    z3_feat = f'{base}_diff_zscore_3m'
    
    row = {'base_feature': base}
    
    if base in wide.columns and diff_feat in wide.columns:
        valid = wide[[base, diff_feat]].dropna()
        if len(valid) > 10:
            row['level_vs_diff'] = round(valid[base].corr(valid[diff_feat]), 3)
    
    if diff_feat in wide.columns and z12_feat in wide.columns:
        valid = wide[[diff_feat, z12_feat]].dropna()
        if len(valid) > 10:
            row['diff_vs_z12'] = round(valid[diff_feat].corr(valid[z12_feat]), 3)
    
    if diff_feat in wide.columns and z3_feat in wide.columns:
        valid = wide[[diff_feat, z3_feat]].dropna()
        if len(valid) > 10:
            row['diff_vs_z3'] = round(valid[diff_feat].corr(valid[z3_feat]), 3)
    
    if z12_feat in wide.columns and z3_feat in wide.columns:
        valid = wide[[z12_feat, z3_feat]].dropna()
        if len(valid) > 10:
            row['z12_vs_z3'] = round(valid[z12_feat].corr(valid[z3_feat]), 3)
    
    if len(row) > 1:
        correlation_analysis.append(row)

corr_df = pd.DataFrame(correlation_analysis)
print('Transformation Correlation Analysis:')
print('=' * 90)
print(corr_df.to_string(index=False))

print(f'\nSummary Statistics:')
for col in ['level_vs_diff', 'diff_vs_z12', 'diff_vs_z3', 'z12_vs_z3']:
    if col in corr_df.columns:
        vals = corr_df[col].dropna().abs()
        if len(vals) > 0:
            print(f'  {col:20s}: mean |r|={vals.mean():.3f}, median={vals.median():.3f}')

Transformation Correlation Analysis:
              base_feature  level_vs_diff  diff_vs_z12  diff_vs_z3  z12_vs_z3
               AHE_Private          0.072        0.701         NaN        NaN
           AWH_All_Private          0.236          NaN       0.823        NaN
         AWH_Manufacturing            NaN          NaN         NaN      0.774
    CB_Consumer_Confidence            NaN          NaN       0.684        NaN
       Challenger_Job_Cuts          0.393        0.729       0.557      0.817
          Empire_State_Mfg          0.416        0.892       0.725      0.824
            Housing_Starts          0.122        0.200       0.106      0.809
   ISM_Manufacturing_Index          0.214          NaN       0.698        NaN
ISM_NonManufacturing_Index            NaN          NaN         NaN      0.818
     Industrial_Production          0.032        0.584       0.381      0.692
              Retail_Sales          0.579        0.217         NaN        NaN
        UMich_Expectations 

## 5. Cross-Series Correlation Analysis

Check for redundancy across different economic indicators.

In [38]:
# Key cross-series pairs to check
cross_pairs = [
    ('AWH_All_Private', 'AWH_Manufacturing', 'AWH All vs Manufacturing'),
    ('ISM_Manufacturing_Index', 'ISM_NonManufacturing_Index', 'ISM Mfg vs Non-Mfg'),
    ('ISM_Manufacturing_Index', 'Empire_State_Mfg', 'ISM vs Empire State'),
    ('ISM_Manufacturing_Index', 'Industrial_Production', 'ISM vs Industrial Prod'),
    ('CB_Consumer_Confidence', 'UMich_Expectations', 'CB Confidence vs UMich'),
    ('AHE_Private', 'AWH_All_Private', 'Earnings vs Hours'),
    ('Housing_Starts', 'Industrial_Production', 'Housing vs Industrial'),
    ('Retail_Sales', 'CB_Consumer_Confidence', 'Retail vs Confidence'),
    ('Retail_Sales', 'Industrial_Production', 'Retail vs Industrial'),
]

print('Cross-Series Level Correlations:')
print('=' * 70)
for feat1, feat2, label in cross_pairs:
    if feat1 in wide.columns and feat2 in wide.columns:
        valid = wide[[feat1, feat2]].dropna()
        if len(valid) > 10:
            r = valid[feat1].corr(valid[feat2])
            flag = ' ** HIGH' if abs(r) > 0.9 else (' * moderate' if abs(r) > 0.7 else '')
            print(f'  {label:40s} r = {r:+.3f} (n={len(valid)}){flag}')
    else:
        missing = [f for f in [feat1, feat2] if f not in wide.columns]
        print(f'  {label:40s} MISSING: {missing}')

# Also check diff correlations for same pairs
print(f'\nCross-Series Diff Correlations:')
print('=' * 70)
for feat1, feat2, label in cross_pairs:
    d1, d2 = f'{feat1}_diff', f'{feat2}_diff'
    if d1 in wide.columns and d2 in wide.columns:
        valid = wide[[d1, d2]].dropna()
        if len(valid) > 10:
            r = valid[d1].corr(valid[d2])
            flag = ' ** HIGH' if abs(r) > 0.9 else (' * moderate' if abs(r) > 0.7 else '')
            print(f'  {label:40s} r = {r:+.3f} (n={len(valid)}){flag}')

Cross-Series Level Correlations:
  AWH All vs Manufacturing                 r = +0.558 (n=237)
  ISM Mfg vs Non-Mfg                       r = +0.689 (n=342)
  ISM vs Empire State                      r = +0.597 (n=294)
  ISM vs Industrial Prod                   r = -0.021 (n=935)
  CB Confidence vs UMich                   MISSING: ['CB_Consumer_Confidence']
  Earnings vs Hours                        r = +0.333 (n=236)
  Housing vs Industrial                    r = -0.215 (n=800)
  Retail vs Confidence                     MISSING: ['CB_Consumer_Confidence']
  Retail vs Industrial                     r = +0.076 (n=406)

Cross-Series Diff Correlations:
  ISM vs Empire State                      r = +0.140 (n=293)
  ISM vs Industrial Prod                   r = +0.267 (n=934)
  Earnings vs Hours                        r = -0.090 (n=235)
  Housing vs Industrial                    r = +0.103 (n=799)
  Retail vs Confidence                     r = +0.053 (n=405)
  Retail vs Industrial          

## 6. Within-Group VIF Analysis

For each base series, compute VIF across its 4 transformation variants.

In [39]:
all_vif_results = []

for group_name, features in SERIES_GROUPS.items():
    print(f'\n{"="*60}')
    print(f'{group_name} ({len(features)} features)')
    print(f'{"="*60}')
    
    vif_df = compute_group_vif(wide, features, group_name)
    if not vif_df.empty:
        all_vif_results.append(vif_df)
        print(vif_df.to_string(index=False))
        
        high_vif = vif_df[vif_df['VIF'] > 5]
        if len(high_vif) > 0:
            print(f'\n  >> {len(high_vif)} features with VIF > 5')
        else:
            print(f'\n  >> All VIF <= 5 (OK)')
    else:
        print('  (not enough features for VIF)')


AHE_Private (4 features)
      group                     feature  VIF
AHE_Private AHE_Private_diff_zscore_12m 3.38
AHE_Private  AHE_Private_diff_zscore_3m 2.21
AHE_Private            AHE_Private_diff 2.00
AHE_Private                 AHE_Private 1.01

  >> All VIF <= 5 (OK)

AWH_All_Private (3 features)
          group                        feature  VIF
AWH_All_Private           AWH_All_Private_diff 3.20
AWH_All_Private AWH_All_Private_diff_zscore_3m 3.11
AWH_All_Private                AWH_All_Private 1.06

  >> All VIF <= 5 (OK)

AWH_Manufacturing (3 features)
            group                           feature  VIF
AWH_Manufacturing AWH_Manufacturing_diff_zscore_12m  2.5
AWH_Manufacturing  AWH_Manufacturing_diff_zscore_3m  2.5
AWH_Manufacturing                 AWH_Manufacturing  1.0

  >> All VIF <= 5 (OK)

CB_Consumer_Confidence (2 features)
                 group                               feature  VIF
CB_Consumer_Confidence           CB_Consumer_Confidence_diff 1.88
CB_Consume

## 7. Cross-Group VIF Pruning

Run iterative VIF pruning across ALL features to eliminate cross-series redundancy.

In [40]:
# Use all features, forward-fill NaN for series with different start dates
all_features = list(wide.columns)
print(f'Starting with {len(all_features)} features')

# Filter to period with good data coverage
wide_recent = wide.loc[wide.index >= '2005-01-01'].ffill()
print(f'Using 2005+ data: {len(wide_recent)} rows')

# Check NaN coverage
nan_pct = wide_recent.isna().mean()
high_nan = nan_pct[nan_pct > 0.3]
if len(high_nan) > 0:
    print(f'\nDropping {len(high_nan)} features with >30% NaN in 2005+:')
    for f, pct in high_nan.items():
        print(f'  {f}: {pct*100:.1f}% NaN')
    all_features = [f for f in all_features if nan_pct.get(f, 1.0) <= 0.3]
    print(f'Remaining: {len(all_features)} features')

print(f'\nRunning iterative VIF pruning (threshold=5)...')
final_survivors, final_removed = iterative_vif_pruning(
    wide_recent, all_features, threshold=50.0
)

print(f'\nFinal feature set: {len(final_survivors)} features')
print(f'Removed: {len(final_removed)} features')

Starting with 41 features
Using 2005+ data: 252 rows

Running iterative VIF pruning (threshold=5)...

Final feature set: 41 features
Removed: 0 features


## 8. Final VIF Verification & Summary

In [41]:
print('FINAL RECOMMENDED FEATURE SET')
print('=' * 60)
print(f'\nTotal features: {len(final_survivors)}')

# Categorize by transformation type
levels, diffs, z12s, z3s = [], [], [], []
for f in sorted(final_survivors):
    if f.endswith('_diff_zscore_12m'):
        z12s.append(f)
    elif f.endswith('_diff_zscore_3m'):
        z3s.append(f)
    elif f.endswith('_diff'):
        diffs.append(f)
    else:
        levels.append(f)

print(f'\nBy transformation type:')
print(f'  Level:           {len(levels)}')
print(f'  MoM diff:        {len(diffs)}')
print(f'  Diff z-score 12m: {len(z12s)}')
print(f'  Diff z-score 3m:  {len(z3s)}')

# Show by base series
for base in BASE_SERIES:
    feats = [f for f in final_survivors if f == base or f.startswith(f'{base}_diff')]
    if feats:
        transforms = []
        for f in feats:
            if f == base: transforms.append('level')
            elif f.endswith('_diff_zscore_12m'): transforms.append('z12')
            elif f.endswith('_diff_zscore_3m'): transforms.append('z3')
            elif f.endswith('_diff'): transforms.append('diff')
        print(f'\n  {base}: {transforms}')

FINAL RECOMMENDED FEATURE SET

Total features: 41

By transformation type:
  Level:           11
  MoM diff:        9
  Diff z-score 12m: 9
  Diff z-score 3m:  12

  AHE_Private: ['level', 'diff', 'z12', 'z3']

  AWH_All_Private: ['level', 'diff', 'z3']

  AWH_Manufacturing: ['level', 'z12', 'z3']

  CB_Consumer_Confidence: ['diff', 'z3']

  Challenger_Job_Cuts: ['level', 'diff', 'z12', 'z3']

  Empire_State_Mfg: ['level', 'diff', 'z12', 'z3']

  Housing_Starts: ['level', 'diff', 'z12', 'z3']

  ISM_Manufacturing_Index: ['level', 'diff', 'z3']

  ISM_NonManufacturing_Index: ['level', 'z12', 'z3']

  Industrial_Production: ['level', 'diff', 'z12', 'z3']

  Retail_Sales: ['level', 'diff', 'z12', 'z3']

  UMich_Expectations: ['level', 'z12', 'z3']


In [42]:
# Final VIF verification
print('\nFinal VIF Verification (should all be <= 10):')
print('=' * 60)
final_vif = compute_group_vif(wide_recent, final_survivors, 'final')
if not final_vif.empty:
    print(final_vif.to_string(index=False))
    max_vif = final_vif['VIF'].max()
    print(f'\nMax VIF: {max_vif:.2f} {"(PASS)" if max_vif <= 10 else "(FAIL)"}')


Final VIF Verification (should all be <= 10):
group                                    feature  VIF
final                 ISM_NonManufacturing_Index 9.41
final                           AHE_Private_diff 9.19
final           Empire_State_Mfg_diff_zscore_12m 8.99
final                      Empire_State_Mfg_diff 7.50
final                 Industrial_Production_diff 6.80
final                          AWH_Manufacturing 6.79
final                    ISM_Manufacturing_Index 5.89
final        Challenger_Job_Cuts_diff_zscore_12m 5.52
final                AHE_Private_diff_zscore_12m 5.43
final                   Challenger_Job_Cuts_diff 5.17
final                       AWH_All_Private_diff 5.07
final ISM_NonManufacturing_Index_diff_zscore_12m 4.95
final      Industrial_Production_diff_zscore_12m 4.85
final               ISM_Manufacturing_Index_diff 4.69
final                        Challenger_Job_Cuts 4.62
final             Housing_Starts_diff_zscore_12m 4.51
final                              

In [43]:
# Removal summary
print('\nREMOVAL SUMMARY')
print('=' * 60)
if final_removed:
    for feat, vif in final_removed:
        print(f'  Removed: {feat:50s} VIF={vif}')
print(f'\nStarted with:  {len(wide.columns)} features')
print(f'Removed:       {len(wide.columns) - len(final_survivors)} features')
print(f'Final set:     {len(final_survivors)} features')


REMOVAL SUMMARY

Started with:  41 features
Removed:       0 features
Final set:     41 features


In [44]:
# Output the SELECTED_FEATURES set for load_unifier_data.py
print('\n# Copy this into load_unifier_data.py:')
print('SELECTED_FEATURES = {')
for f in sorted(final_survivors):
    print(f"    '{f}',")
print('}')


# Copy this into load_unifier_data.py:
SELECTED_FEATURES = {
    'AHE_Private',
    'AHE_Private_diff',
    'AHE_Private_diff_zscore_12m',
    'AHE_Private_diff_zscore_3m',
    'AWH_All_Private',
    'AWH_All_Private_diff',
    'AWH_All_Private_diff_zscore_3m',
    'AWH_Manufacturing',
    'AWH_Manufacturing_diff_zscore_12m',
    'AWH_Manufacturing_diff_zscore_3m',
    'CB_Consumer_Confidence_diff',
    'CB_Consumer_Confidence_diff_zscore_3m',
    'Challenger_Job_Cuts',
    'Challenger_Job_Cuts_diff',
    'Challenger_Job_Cuts_diff_zscore_12m',
    'Challenger_Job_Cuts_diff_zscore_3m',
    'Empire_State_Mfg',
    'Empire_State_Mfg_diff',
    'Empire_State_Mfg_diff_zscore_12m',
    'Empire_State_Mfg_diff_zscore_3m',
    'Housing_Starts',
    'Housing_Starts_diff',
    'Housing_Starts_diff_zscore_12m',
    'Housing_Starts_diff_zscore_3m',
    'ISM_Manufacturing_Index',
    'ISM_Manufacturing_Index_diff',
    'ISM_Manufacturing_Index_diff_zscore_3m',
    'ISM_NonManufacturing_Index',
    